In [1]:
!nvidia-smi

Tue Jun 11 19:59:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.99                 Driver Version: 555.99         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   51C    P0             24W /  100W |     930MiB /   6144MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

In [3]:
import pandas as pd
import numpy as np
import torch
import transformers
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
%pwd

'e:\\Projects for portfolio\\Exoplanet Chatbot\\research'

In [10]:
import os
os.chdir("../")
%pwd

'e:\\Projects for portfolio\\Exoplanet Chatbot'

In [11]:
finetuning_dataset = pd.read_csv('artifacts\Instruction Q-A pair Exoplanet Data.csv')

In [12]:
finetuning_dataset.shape

(101484, 3)

In [15]:
# Function to generate prompts for finetuning our model
def generate_prompt(data_point):
    """Generate input text based on a prompt, task instruction, context info, and answer.

    :param data_point: dict: Data point
    :return: str: tokenized prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n'
    instruction = data_point['instruction']
    context = data_point['input']
    response = data_point['output']

    # If context is provided
    if context:
        text = f"""<start_of_turn>user {prefix_text} {instruction} here is the context: {context} <end_of_turn>\n<start_of_turn>model {response} <end_of_turn>"""
    # If context is not provided
    else:
        text = f"""<start_of_turn>user {prefix_text} {instruction} <end_of_turn>\n<start_of_turn>model {response} <end_of_turn>"""

    return text

In [16]:
# add the "prompt" column in the dataset
finetuning_dataset['prompt'] = finetuning_dataset.apply(generate_prompt, axis=1)

In [17]:
finetuning_dataset.head()

,instruction,input,output,prompt
0,What is the number of stars of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of stars of 11 Com b is 2.,<start_of_turn>user Below is an instruction th...
1,What is the number of planets of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of planets of 11 Com b is 1.,<start_of_turn>user Below is an instruction th...
2,What is the number of moons of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of moons of 11 Com b is 0.,<start_of_turn>user Below is an instruction th...
3,What is the binary system of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The binary system of 11 Com b is Binary System.,<start_of_turn>user Below is an instruction th...
4,What is the discovery method of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The discovery method of 11 Com b is Radial Vel...,<start_of_turn>user Below is an instruction th...


In [18]:
# Importing the instruction finetuned Gemma - 2 billion parameter model
model_id = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True, padding_side="left")

c:\Users\arany\anaconda3\envs\exochat\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C'), WindowsPath('/Users/arany/anaconda3/envs/exochat/lib')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/matplotlib_inline.backend_inline'), WindowsPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=121, Highest Compute Capability: 8.6.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/bl

c:\Users\arany\anaconda3\envs\exochat\lib\site-packages\bitsandbytes\cuda_setup\main.py:167: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
c:\Users\arany\anaconda3\envs\exochat\lib\site-packages\bitsandbytes\cuda_setup\main.py:167: UserWarning: C:\Users\arany\anaconda3\envs\exochat did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
# Converting the dataframe to a dataset
prompt_dataset = Dataset.from_pandas(finetuning_dataset)

In [ ]:
# Shuffling the dataset and Tokenization
prompt_dataset = prompt_dataset.shuffle(seed=1234)  # Shuffle dataset here
prompt_dataset = prompt_dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [ ]:
# Splitting the data into training and testing datasets
prompt_dataset = prompt_dataset.train_test_split(test_size=0.2)
train_data = prompt_dataset["train"]
test_data = prompt_dataset["test"]

In [ ]:
# LoRa config libraries
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
print(model)

NameError: name 'model' is not defined

In [ ]:
# Linear layer modules which will be modified/trained during LoRa finetuning
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

In [ ]:
# LoRa Config
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64, #  Always keep it 2 times the lora_alpha
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
# Total Trainable parameters
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

In [ ]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100, # Will be later changing to 100
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
# Training the model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
new_model = "gemma-Code-Instruct-Finetune-test1" #Name of the model you will be pushing to huggingface model hub

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("Gemma_exochatbot_finetuned100steps",safe_serialization=True)
tokenizer.save_pretrained("Gemma_exochatbot_finetuned100steps_tokenizer")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub (learn about this later)
#merged_model.push_to_hub(new_model, use_temp_dir=False)
#tokenizer.push_to_hub(new_model, use_temp_dir=False)

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [ ]:
result = get_completion(query="What is the planet density of 11 Com b?", model=merged_model, tokenizer=tokenizer)
print(result)